In [1]:
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#!pip install tensorflow_hub
import tensorflow_hub as hub
import numpy as np
import shutil
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import tensorboard
import os
import pandas as pd
import math

Model 3: Transfer Learning: EfficientNet

The original image sizes used for every version of EfficientNet are:

EfficientNetB0 - (224, 224, 3)
EfficientNetB1 - (240, 240, 3)
EfficientNetB2 - (260, 260, 3)
EfficientNetB3 - (300, 300, 3)
EfficientNetB4 - (380, 380, 3)
EfficientNetB5 - (456, 456, 3)
EfficientNetB6 - (528, 528, 3)
EfficientNetB7 - (600, 600, 3)

We are however not bound by this and can use a smaller size if we want.

Reference: https://www.kaggle.com/code/carlolepelaars/efficientnetb5-with-keras-aptos-2019/notebook

In [2]:
batch_size = 32
img_height = 224
img_width = 224

In [4]:
dataset = 'Data/Processed_Data/train'
dataset_path = os.listdir(dataset)

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

['train_ak_aug', 'train_bcc_aug', 'train_bkl_aug', 'train_df_aug', 'train_mel_aug', 'train_nv_aug', 'train_scc_aug', 'train_vasc_aug']
Types of classes labels found:  8


In [5]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# create a directory to save the files
create_dir("models")

In [6]:
# load the augmented data from the local data directory:

train_dir = "Data/Processed_Data/train"
test_dir = "Data/Processed_Data/test"
valid_dir = "Data/Processed_Data/valid_processed"

datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.2, data_format="channels_last", dtype=tf.float32) # The imported data will br normalized here.
test_datagen = ImageDataGenerator(rescale = 1./255, data_format="channels_last", dtype=tf.float32)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    shuffle = True,
    seed=123,subset = 'training') # set as training data

validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    shuffle = True,
    seed=123,subset = 'validation') # set as validation data

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    shuffle = True,
    seed=123,) # set as testing data

Found 57680 images belonging to 8 classes.
Found 14416 images belonging to 8 classes.
Found 3799 images belonging to 8 classes.


In [8]:
# Nasnet model
model = Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2",
               trainable=True),
    layers.Dense(8, activation='softmax'),
])

model.build([None, img_height, img_width, 3])  # Batch input shape.

In [9]:
METRICS = [
    keras.metrics.Accuracy(name= "accuracy"),
    keras.metrics.Precision(name = "precision"),
    keras.metrics.Recall(name = 'recall'),
    keras.metrics.AUC(name = 'auc'),
]
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS,)


r = model.fit(
    train_generator,
    epochs = 2,
    steps_per_epoch=train_generator.samples//batch_size,
    validation_data = validation_generator,
    validation_steps=validation_generator.samples//batch_size,
)

Epoch 1/2


ResourceExhaustedError: Graph execution error:

2 root error(s) found.
  (0) RESOURCE_EXHAUSTED:  OOM when allocating tensor with shape[32,1152,7,7] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node tpu_batch_normalization_1/FusedBatchNormV3}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[assert_less_equal/Assert/AssertGuard/pivot_f/_2255/_81]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

  (1) RESOURCE_EXHAUSTED:  OOM when allocating tensor with shape[32,1152,7,7] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node tpu_batch_normalization_1/FusedBatchNormV3}}]]
Hint: If you want 

In [ ]:
#Confution Matrix and Classification Report
Y_pred = model.predict_generator(test_generator, test_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['ak', 'bcc', 'bkl', 'df', 'mel','nv', 'scc', 'vasc']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))
